# Clustering Neighborhoods (Toronto)

## 1st 

We start by importing the required libraries.


In [209]:
import pandas as pd

import numpy as np

Then we read the table directly from Toronto Postal Codes' Wikepiedia page.

In [210]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

table = pd.read_html(url)[0]
table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


And drop the rows whose Borough values are 'Not Assigned'.

In [211]:
to_drop = table[table['Borough'] == 'Not assigned'].index

table.drop(index=to_drop, inplace=True)

In [212]:
table = table.reset_index().drop(columns='index')

In [213]:
table.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Then we check for any potential 'Not assigned' value in the Neighbourhood column.

In [214]:
(table['Neighbourhood'].isin(['Not assigned'])).sum()

0

And print the number of rows and columns in our dataframe.

In [215]:
print(table.shape)

(103, 3)


## 2nd

In [216]:
url = 'https://cocl.us/Geospatial_data'

ll = pd.read_csv(url)

In [217]:
ll.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [218]:
Toronto_table = table.merge(ll, how='left')

In [219]:
Toronto_table.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [220]:
Toronto_table.isna().sum()

Postal Code      0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

## 3rd

To simplify the analysis we will be working only with boroughs containing the word 'Toronto'.

In [221]:
Toronto_table = (Toronto_table[Toronto_table['Borough'].map(lambda x: "Toronto" in x)].reset_index())

In [222]:
Toronto_table.drop(columns='index', inplace=True)

In [223]:
Toronto_table.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Then we import the required libraries.

In [224]:
import json

import requests
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

In [225]:
address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7793879, -79.3046089.


Now we proceed by creating a map of toronto with neighborhoood superimposed on top.

In [226]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [192]:
for lat, lng, borough, neighborhood in zip(Toronto_table['Latitude'], Toronto_table['Longitude'], Toronto_table['Borough'], Toronto_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now, we are going to leverage location data from the Fourquare API, in order to segment the neighborhoods in Toronto. The first step is that of defining our Foursquare Crenditials.

In [231]:
CLIENT_ID = 'HOWKV2SL5WLHW4D4SDEVDG4UQJ1JZIWM3BUKVH12M4FW0CEK'
CLIENT_SECRET = 'KJ44YCASMSD25R25BETVJ1YE0H5APBMOKMEKNLNOCGM4RD3K'
VERSION = '20210304' 
LIMIT = 100
radius = 500

And drop the 'Postal Code' column since we won't need it in the next few steps.

In [228]:
Toronto_table.drop(['Postal Code'], axis=1, inplace=True)

Then, we create a function to get the top 100 venues for all the neighborhoods in our dataframe. Venues will be included if within a 500 meters radius from the center of the neighborhood. 

In [232]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we can proceed by applying the getNearbyVenues() function to our dataset.

In [233]:
Toronto_venues = getNearbyVenues(names=Toronto_table['Neighbourhood'],
                                   latitudes=Toronto_table['Latitude'],
                                   longitudes=Toronto_table['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [366]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [445]:
Toronto_venues.shape

(1606, 7)

We check the number of venues returned for each neighborhood.

In [446]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


And check how many unique venue categories were returned.

In [447]:
len(Toronto_venues['Venue Category'].unique())

237

Now we are ready to implement __our cluster analysis on each neighborhood__ and their respective venues. 
We start by apllying onehot encoding to our cateogrical variable 'Venue Category'.

In [448]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']

Then, we group by 'Neighborhood' and extrapolate the mean occurence of each category in each neighborhood.

In [449]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

We print each neighborhood and the five most common venues for each neighborhood.

In [450]:
n_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(n_top_venues))
    print('\n')

----Berczy Park----
                   venue                   freq
0            Yoga Studio            Yoga Studio
1          Women's Store          Women's Store
2               Wine Bar               Wine Bar
3  Vietnamese Restaurant  Vietnamese Restaurant
4       Video Game Store       Video Game Store


----Brockton, Parkdale Village, Exhibition Place----
                   venue                   freq
0            Yoga Studio            Yoga Studio
1          Women's Store          Women's Store
2               Wine Bar               Wine Bar
3  Vietnamese Restaurant  Vietnamese Restaurant
4       Video Game Store       Video Game Store


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                   venue                   freq
0            Yoga Studio            Yoga Studio
1          Women's Store          Women's Store
2               Wine Bar               Wine Bar
3  Vietnamese Restaurant  Vietnamese Restaurant
4       Video 

Then with proceed by enclosing the results into a pandas dataframe.

We write a functino to sort the venues in descending order.

In [460]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

And create a new dataframe displaying the top 10 venues for each neighborhood.

In [461]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Farmers Market,Restaurant,Pharmacy,Cheese Shop,Beer Bar,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Smoke Shop,Light Rail Station,Burrito Place,Farmers Market,Fast Food Restaurant,Butcher,Restaurant,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Harbor / Marina,Coffee Shop,Sculpture Garden,Boat or Ferry,Airport Terminal,Airport Gate,Airport Food Court
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Yoga Studio


Now we are ready to cluster our Toronoto Neighborhood(in the boroughs whose names include the word Toronto).

We will run a k-means algorithm to do so.

In [462]:
kclusters = 5

Toronto_clustering = Toronto_grouped.drop('Neighborhood', 1)

kmeans_fit = KMeans(kclusters, random_state=0).fit(Toronto_clustering)

Then we visualize the labels produced by running k-means.

In [463]:
kmeans_fit.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

We now create a dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [464]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans_fit.labels_)

In [ ]:
Toronto_to_merge = Toronto_table

In [476]:
Toronto_merged = pd.merge(Toronto_to_merge, neighborhoods_venues_sorted.set_index('Neighborhood'), left_on=['Neighbourhood'], 
             right_on= ['Neighborhood'], how='left')

Now we can visualize the resulting clusters. 

In [478]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters